In [1]:
import snap
from operator import itemgetter
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from random import choice
from pandas import datetools
import datetime
import numpy as np
from __future__ import division

### For whole network run this:

In [2]:
edges = pd.read_csv("data/edges_filtered.csv", ',')
G = nx.from_pandas_edgelist(edges, '0', '1', True, nx.DiGraph())
metadata_filtered = pd.read_csv('data/metadata_filtered.csv',' ')
metadata_filtered['Timestamp']=metadata_filtered['created_at'].astype('datetime64')
del metadata_filtered['Unnamed: 0']
metadata_filtered=metadata_filtered.rename(index=str, columns={"user_id": "id"})

### For ego-networks run this:

In [2]:
ego = 1

In [3]:
ego_edges = pd.read_csv('data/Ego '+str(ego)+'/ego_edges'+str(ego)+'.csv',',', index_col=None, comment='#')
ego_nodes = pd.read_csv('data/Ego '+str(ego)+'/ego_nodes'+str(ego)+'.csv',',', index_col=None, comment='#')
metadata_filtered = pd.read_csv('data/Ego '+str(ego)+'/ego_meta'+str(ego)+'.csv', ',')
metadata_filtered['Timestamp']=metadata_filtered['Timestamp'].astype('datetime64')
metadata_filtered=metadata_filtered.sort_values(by=['Timestamp'])
G = nx.from_pandas_edgelist(ego_edges, 'Source', 'Target', True, nx.DiGraph())

### Attribute assortativity

In [8]:
metadata_mode = metadata_filtered.groupby('id')['emotion_category_id'].apply(lambda x: x.value_counts().index[0]).reset_index()

In [29]:
# We populate the graph with the most frequent emotions
for id in G.nodes:
    G.nodes[id]['emotion'] = metadata_mode[metadata_mode['id']==id].emotion_category_id.item()

In [30]:
nodes_attributes = []
for (p, d) in G.nodes(data=True):
    if len(d.values())>0 :
        nodes_attributes.append(p)

In [31]:
attr_assortativity = nx.attribute_assortativity_coefficient(G, 'emotion', nodes_attributes)

In [32]:
attr_assortativity

0.029460145549295522

### Numeric assortativity

In [4]:
metadata_mode = pd.DataFrame(metadata_filtered.groupby('id')['reactions'].sum())
metadata_mode['id'] = metadata_mode.index

In [5]:
for id in G.nodes:
    G.nodes[id]['reactions'] = metadata_mode[metadata_mode['id']==id].reactions.item()

In [6]:
nodes_attributes = []
for (p, d) in G.nodes(data=True):
    if len(d.values())>0 :
        nodes_attributes.append(p)

In [7]:
numeric_asortativity = nx.numeric_assortativity_coefficient(G, 'reactions', nodes_attributes)

In [8]:
numeric_asortativity

-0.16248798167330167

### Degree assortativity

In [3]:
degree_assortativity = nx.degree_assortativity_coefficient(G)

In [4]:
degree_assortativity

-0.12075149552057499

### Pearson assortativity

In [ ]:
pearson_assortativity = nx.degree_pearson_correlation_coefficient(G)

In [ ]:
pearson_assortativity

### Attribute Assortativity by Last emotion from each user

In [ ]:
n_sims=100
assortativity0=[]
for i in np.arange(0,n_sims,1):
    
    vent = metadata_filtered.sample(1)
    idx = metadata_filtered.Timestamp.values <= vent.Timestamp.values
    metadata_last = metadata_filtered[idx]
    idx = metadata_last.groupby(['id'])['Timestamp'].transform(max) == metadata_last['Timestamp']
    metadata_last = metadata_last[idx]

    for id in metadata_last['id']:
        while G.has_node(id):
            G.nodes[id]['emotion'] = metadata_last[metadata_last['id']==id]['emotion_category_id'].item()
    
    nodes_attributes = []

    for (p, d) in G.nodes(data=True):
        if len(d.values())>0 :
            nodes_attributes.append(p)
        
    assortativity1 = nx.attribute_assortativity_coefficient(G, 'emotion', nodes_attributes)
    assortativity0 = np.append(assortativity0, assortativity1)

assortativity_avg = np.mean(assortativity0)

In [ ]:
assortativity_avg